,open,high,low,close,RSI,RSI_BB_UPPER,RSI_BB_MIDDLE,RSI_BB_LOWER,RSI_ST,RSI_LOG,MACD,MACD_ST,ATR,ADX,ADXR,+DI,-DI,SMA20,SMA50,SMA200,BB_UPPER,BB_MIDDLE,BB_LOWER,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,BBANDS_bandwidth,BBANDS_squeeze,STOCH_K,STOCH_D,MON,target
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-02 08:00:00,2507.9,2508.7,2506.3,2508.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0
2019-01-02 08:01:00,2508.7,2512.0,2508.5,2512.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1
2019-01-02 08:02:00,2512.0,2516.8,2511.5,2516.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1
2019-01-02 08:03:00,2516.2,2516.4,2514.9,2514.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0
2019-01-02 08:04:00,2514.9,2515.7,2514.8,2515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-01 05:10:00,4139.0,4139.1,4138.9,4139.0,63.317318,90.325936,65.243983,40.162031,0.015343,4.151094,2.581791,0.000624,1.587505,39.445910,33.351436,34.618393,11.964693,4136.065,4132.588,4121.3620,4145.436505,4136.065,4126.693495,0.001555,-0.000709,-0.002973,0.004532,0,0.011712,0.017319,-0.2,0
2023-04-01 05:11:00,4139.0,4139.3,4138.9,4139.2,64.496931,89.126922,66.356480,43.586039,0.015469,4.159311,2.484512,0.000600,1.502683,40.176747,34.999398,34.910853,11.737201,4136.700,4132.758,4121.4695,4145.059426,4136.700,4128.340574,0.001440,-0.000580,-0.002599,0.004042,1,0.007276,0.012298,0.1,1
2023-04-01 05:12:00,4139.2,4139.5,4139.1,4139.4,65.755455,84.894335,67.993003,51.091670,0.015600,4.167829,2.395938,0.000579,1.423920,40.932578,35.758848,35.213622,11.501690,4137.425,4132.922,4121.5815,4143.858313,4137.425,4130.991687,0.001101,-0.000453,-0.002007,0.003110,1,0.007715,0.008900,-0.5,1


[LightGBM] [Info] Number of positive: 326964, number of negative: 401316
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012500 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7652
[LightGBM] [Info] Number of data points in the train set: 728280, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448954 -> initscore=-0.204899
[LightGBM] [Info] Start training from score -0.204899
Training until validation scores don't improve for 10 rounds
[100]	training's binary_error: 0.175578	valid_1's binary_error: 0.177657
[200]	training's binary_error: 0.161637	valid_1's binary_error: 0.163905
[300]	training's binary_error: 0.155409	valid_1's binary_error: 0.158273
[400]	training's binary_error: 0.15159	valid_1's binary_error: 0.155269
[500]	training's binary_error: 0.148919	valid_1's binary_error: 0.153271
[600]	train

[LightGBM] [Info] Number of positive: 327252, number of negative: 401028
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7652
[LightGBM] [Info] Number of data points in the train set: 728280, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.449349 -> initscore=-0.203301
[LightGBM] [Info] Start training from score -0.203301
Training until validation scores don't improve for 10 rounds
[100]	training's binary_error: 0.174909	valid_1's binary_error: 0.177678
[200]	training's binary_error: 0.160847	valid_1's binary_error: 0.164404
[300]	training's binary_error: 0.154586	valid_1's binary_error: 0.159222
[400]	training's binary_error: 0.151053	valid_1's binary_error: 0.155988
[500]	training's binary_error: 0.148424	valid_1's binary_error: 0.153904
[600]	trai

Mean accuracy: 0.8491733948481353


Model saved to model/simple_1m_model.pkl


In [2]:
import os
import pandas as pd
import numpy as np
import talib
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

pd.set_option('display.max_columns', None)

def load_data(file_names):
    dfs = []
    for file_name in file_names:
        df = pd.read_csv(file_name, index_col="date")
        dfs.append(df)
    return dfs

def feature_engineering(df):
    open = df['open'].values
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values
    # volume = df['volume'].values
    hilo = (high + low) / 2

    rsi = talib.RSI(close, timeperiod=8) # default = 14
    df['RSI'] = rsi
    df['RSI_BB_UPPER'], df['RSI_BB_MIDDLE'], df['RSI_BB_LOWER'] = talib.BBANDS(rsi, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    
    df['RSI_ST'] = talib.RSI(close)/close
    df['RSI_LOG'] = log_transform_feature(talib.RSI(close))
    df['MACD'], _, _ = talib.MACD(close)
    df['MACD_ST'], _, _ = talib.MACD(close)/close
    df['ATR'] = talib.ATR(high, low, close)
    
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['+DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['-DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    
    df['SMA20'] = talib.SMA(close, timeperiod=20)
    df['SMA50'] = talib.SMA(close, timeperiod=50)
    df['SMA200'] = talib.SMA(close, timeperiod=200)
    
    df['BB_UPPER'], df['BB_MIDDLE'], df['BB_LOWER'] = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] = (df['BB_UPPER'] - hilo) / close
    df['BBANDS_middleband'] = (df['BB_MIDDLE'] - hilo) / close
    df['BBANDS_lowerband'] = (df['BB_LOWER'] - hilo) / close

    df['STOCH_K'], df['STOCH_D'] = talib.STOCH(high, low, close)/close
    df['MON'] = talib.MOM(close, timeperiod=5)

    df = df.fillna(method='ffill')

    return df

def log_transform_feature(X):
    X[X <= 0] = np.finfo(float).eps
    return np.log(X)

def create_label(df, lookbehind=1):
    df['target'] = (df['close'] > df['close'].shift(lookbehind)).astype(int)
    df = df.fillna(method='ffill')
    return df

def plot_learning_curve(evals_result):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(np.arange(len(evals_result['training']['binary_error'])),
            evals_result['training']['binary_error'], label='Training')
    ax.plot(np.arange(len(evals_result['valid_1']['binary_error'])),
            evals_result['valid_1']['binary_error'], label='Validation')
    ax.set_title('Learning Curve')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Binary Error')
    ax.legend()
    plt.show()

def plot_feature_importance(importances, feature_names):
    importance = pd.DataFrame({"Feature": feature_names,
                               "Importance": importances})
    importance.sort_values(by="Importance", ascending=False, inplace=True)
    plt.figure(figsize=(15, 30))
    sns.barplot(x="Importance", y="Feature", data=importance)
    plt.title("Feature Importance")
    plt.tight_layout()
    plt.show()

def train_and_evaluate(df, n_splits=2):
    features = df.drop('target', axis=1)
    labels = df['target']

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []
    feature_importances = []

    for train_index, test_index in kf.split(features):
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        params = {
            'objective': 'binary',
            'metric': 'binary_error',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9
        }

        evals_result = {}

        model = lgb.train(
            params=params,
            train_set=train_data,
            valid_sets=[train_data, test_data],
            num_boost_round=10000,
            callbacks=[
                lgb.callback.early_stopping(10),
                lgb.callback.log_evaluation(period=100),
                lgb.callback.record_evaluation(evals_result)
            ],
        )

        y_pred = model.predict(X_test)
        y_pred = np.round(y_pred).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        print(f"Accuracy: {accuracy}")
        print(classification_report(y_test, y_pred))
        
        plot_learning_curve(evals_result)
        feature_importances.append(model.feature_importance())
    
    mean_accuracy = np.mean(accuracies)
    print(f"Mean accuracy: {mean_accuracy}")

    mean_importance = np.mean(feature_importances, axis=0)
    plot_feature_importance(mean_importance, features.columns)

    return model, evals_result

if __name__ == "__main__":
    file_names = ["combined_data.csv"]
    dfs = load_data(file_names)

    # Assuming you have only one DataFrame in the list
    combined_df = dfs[0]
    # Convert the index to datetime
    combined_df.index = pd.to_datetime(combined_df.index)

    # Apply feature engineering and label creation directly on the original data
    combined_df = feature_engineering(combined_df)
    combined_df = create_label(combined_df, 15)
    display(combined_df)

    model, evals_result = train_and_evaluate(combined_df)

    model_path = os.path.join("model", "simple_1m_model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)

    print(f"Model saved to {model_path}")